In [13]:
import matplotlib.pyplot as plt
import numpy as np
from HelperAndMechanics import *
from matplotlib import cm

# Set Palatino as the default font
font = {'family': 'serif', 'serif': ['Palatino'], 'size': 20}
plt.rc('font', **font)
plt.rc('text', usetex=True)

import jax.numpy as jnp
from jax import jit
from jax.scipy.signal import convolve2d
from jax.flatten_util import ravel_pytree
from jax.experimental.ode import odeint
from jax import tree_util

import numpy as np
import os
import matplotlib.pyplot as plt

from adoptODE import train_adoptODE, simple_simulation, dataset_adoptODE

In [10]:


def define_AP(**kwargs_sys):
    disc_x, disc_y = kwargs_sys['size'], kwargs_sys['size']
    dx, dy = kwargs_sys['spacing'], kwargs_sys['spacing']
    # N_sys = kwargs_sys['N_sys']
    # padding = kwargs_sys['padding']

    def d_dx(f):
        return jnp.concatenate((jnp.zeros(
            (1, disc_x)), f[1:] - f[:-1], jnp.zeros((1, disc_x))),
                            axis=0)

    def d_dy(f):
        return jnp.concatenate((jnp.zeros(
            (disc_y, 1)), f[:, 1:] - f[:, :-1], jnp.zeros((disc_y, 1))),
                            axis=1)

    kernel = np.array([[1, 4, 1], [4, -20.0, 4], [1, 4, 1]]) / (
      dx * dy * 6)  #np.array([[0,1,0],[1,-4,1.],[0,1,0]])/(dx*dy)

    def laplace(f):  #laplace of scalar
        f_ext = jnp.concatenate((f[0:1], f, f[-1:]), axis=0)
        f_ext = jnp.concatenate((f_ext[:, 0:1], f_ext, f_ext[:, -1:]), axis=1)
        return convolve2d(f_ext, kernel, mode='valid')

    def epsilon(u,v,rp):
        return rp['eps0']+rp['mu1']*v/(u+rp['mu2'])

    def eom(y, t, params, iparams, exparams):
            p=params
            u=y['u']
            v=y['v']
            dudt = p['D']*laplace(u)-(10.0**p['logk'])*u*(u-p['a'])*(u-1) - u*v
            dvdt = epsilon(u,v,p)*(-v-(10.0**p['logk'])*u*(u-p['a']-1))
            return {'u':dudt, 'v':dvdt}
        
    def gen_params():
        return {'D':1.5,'a':0.06,'logk':1.0,'eps0':0.001,'mu1':0.2,'mu2':0.3}, {}, {}

    def loss(ys, params, iparams, exparams, targets):
        flat_fit = ys['u']
        flat_target = targets['u']
        return jnp.nanmean((flat_fit - flat_target)**2)  
            
    return eom, loss, gen_params, None, {}

In [11]:
"""
    Reads in necessary parameters from config.ini
"""
N,size,[] = read_config([],mode = 'chaos')

# Load from HDF5
with h5py.File('../data/SpringMassModel/MechanicalData/data_eta_05_uv.h5', 'r') as f:
    dA = f['dA'][:]
    u = f['u'][:]
    T = f['T'][:]
    v = f['v'][:]
    f.close()
N = T.shape[0]

NameError: name 'read_config' is not defined